Read and import all datasets 
In this chapter we are going to import and clean the datasets needed for this project

The dataset was gotten nigerian university which records or have a detailed information about network traffic data, average_transmit_bmp, average_recieve_bmp
date time

In [ ]:
import os # this libary or module allows me to have acess to my filesystem
import pandas as pd #dataframe liabray 


# The read_and_merge_xlsx_files function allows us loop through the file directiory where we have all the individual raw dataset

In [ ]:
def read_and_merge_xlsx_files(directory):
    # Initialize an empty list to store the dataframes
    dataframes = []
    files = []
    # Loop through all the files in the directory
    for file_name in os.listdir(directory):
        print(file_name)
        # Check if the file is an .xlsx file
        if file_name.endswith('.xlsx'):
            # Construct the full file path
            file_path = os.path.join(directory, file_name)
            xls = pd.ExcelFile(file_path)
            # Read the Excel file
            if 'ChartData' in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name='ChartData', skiprows=3)
            else:
                # Load the first sheet as a fallback
                df = pd.read_excel(xls, sheet_name=xls.sheet_names[0], skiprows=3)
            # df = pd.read_excel(file_path, sheet_name='ChartData', skiprows=3)
            # Append the dataframe to the list
            dataframes.append(df)
           
            files.append(file_name)
            print('done')
    
    # Merge all dataframes in the list into a single dataframe
    merged_df = pd.concat(dataframes, ignore_index=True)
    
    return merged_df

In [ ]:
directory = './raw_datasets'
raw_data = read_and_merge_xlsx_files(directory)

In [ ]:
# Rename colunms
raw_data

In [ ]:
# Select Important Columns
df = raw_data[['DATE / TIME', 'Unnamed: 1',
      'Average Receive bps INTERNET-EDGE-ROUTER - GigabitEthernet0/2.1 · Connection-GLO-2STM-Circuit',
      'Average Transmit bps INTERNET-EDGE-ROUTER - GigabitEthernet0/2.1 · Connection-GLO-2STM-Circuit']]

In [ ]:
# Rename columns for clarity
df.columns = ['Date', 'Time', 'Average_Receive_bps', 'Average_Transmit_bps']

In [ ]:
# Drop any rows where all data is NaN
df = df.dropna(how='all')

In [ ]:
df.isnull().sum()

In [ ]:
# Drop all row where date and time is null
df = df.dropna(subset=['Date', 'Time'])

In [ ]:
df.isnull().sum()

In [ ]:
# Convert Date and Time to string before concatenating
df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))

In [ ]:
# reorder the date for better understanding
cleaned_data = df[['Datetime', 'Average_Receive_bps', 'Average_Transmit_bps']]

In [ ]:
cleaned_data_filled = cleaned_data.fillna(method='ffill')
cleaned_data_filled.dtypes

In [ ]:
cleaned_data_filled.isnull().sum()

In [ ]:
total_duplicated_rows = cleaned_data_filled.duplicated().sum()
print(f"Total duplicated rows: {total_duplicated_rows}")

In [ ]:
# drop duplicates in place without creating a new DataFrame:
cleaned_data_filled.drop_duplicates(inplace=True)
print("Duplicated rows have been removed.")
print(cleaned_data_filled.duplicated().sum())

In [ ]:
cleaned_data_filled.to_csv('./datasets/cleaned_data.csv', index=False)

In [ ]:
cleaned_data_filled.shape

In [ ]:
# Handling outliers in the dataset (e.g., capping)
cap_receive = cleaned_data_filled['Average_Receive_bps'].quantile(0.99)
cleaned_data_filled['Average_Receive_bps'] = cleaned_data_filled['Average_Receive_bps'].clip(upper=cap_receive)
print(cap_receive)

In [ ]:
cleaned_data_filled['Day'] = cleaned_data_filled['Datetime'].dt.dayofweek
cleaned_data_filled['Hour'] = cleaned_data_filled['Datetime'].dt.hour
cleaned_data_filled['Minute'] = cleaned_data_filled['Datetime'].dt.minute

In [ ]:
# reorder the date for better understanding
training_data = cleaned_data_filled[['Day', 'Hour', 'Minute','Datetime', 'Average_Receive_bps', 'Average_Transmit_bps']]

In [ ]:
cleaned_data_filled.to_csv('./datasets/cleaned_data_wihtout_outliners.csv', index=False)
training_data.to_csv('./datasets/final_train_dataset.csv', index=False)